In [1]:
### 2. Настройка логирования
LOG_FILE = 'grandchildren_optimization_debug.log'

# --- Правильное пересоздание логгера ---
# 1. Получаем корневой логгер
logger = logging.getLogger()

# 2. Закрываем и удаляем существующие обработчики
for handler in logger.handlers[:]:
    handler.close()
    logger.removeHandler(handler)

# 3. Удаляем файл, если он существует (теперь он не занят)
if os.path.exists(LOG_FILE):
    try:
        os.remove(LOG_FILE)
    except (PermissionError, OSError) as e:
        print(f"Не удалось удалить старый лог-файл: {e}")


# 4. Конфигурируем логгер заново
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler(sys.stdout) 
    ]
)

logging.info("Логгер инициализирован.")


NameError: name 'logging' is not defined

In [ ]:
### 3. Инициализация системы
pendulum = PendulumSystem()
spore_tree_config = SporeTreeConfig(
    initial_position=np.array([np.pi/2, 0.0]),
    dt_base=integ_config['step_size'],
    dt_grandchildren_factor=0.1,
    show_debug=False
)

tree = SporeTree(pendulum, spore_tree_config)
evaluator = TreeEvaluator(tree)

num_children = 4
num_grandchildren_per_child = 2
num_total_grandchildren = num_children * num_grandchildren_per_child
# В новой задаче мы оптимизируем только 'dt' внуков
num_params = num_total_grandchildren

logging.info(f"Система инициализирована. Количество оптимизируемых параметров (только внуки): {num_params}")



In [ ]:
### 4. Целевая функция и цикл оптимизации

@dataclass
class OptimizationState:
    iteration: int = 0
    history: list = field(default_factory=list)

opt_state = OptimizationState()

def grandchildren_pairing_loss(dt_grandchildren, fixed_dt_children):
    """
    Целевая функция, минимизирующая попарные расстояния между внуками.
    """
    # Собираем полный вектор dt
    dt_all = np.concatenate([fixed_dt_children, dt_grandchildren])
    
    # Строим дерево с текущими параметрами
    evaluator._build_if_needed(dt_all)
    
    # Получаем позиции внуков
    grandchildren_positions = np.array([gc['position'] for gc in tree.grandchildren])
    
    # Вычисляем попарные расстояния
    # Используем pairwise_sqdist, но без Sinkhorn, нам нужна просто сумма расстояний
    dist_matrix = pairwise_sqdist(grandchildren_positions, grandchildren_positions)
    
    # Суммируем верхний треугольник матрицы расстояний (без диагонали),
    # чтобы каждая пара учитывалась один раз.
    loss = np.sum(np.triu(dist_matrix, k=1))
    
    return loss

def callback_function(current_dt_grandchildren):
    """
    Коллбэк для логирования прогресса оптимизации.
    """
    # Для расчета и логирования нам нужен полный вектор dt
    fixed_dt_children = spore_tree_config.get_default_dt_vector()[:num_children]
    
    current_loss = grandchildren_pairing_loss(current_dt_grandchildren, fixed_dt_children)
    
    logging.info(f"Iter {opt_state.iteration}: Loss={current_loss:.6f}")
    
    opt_state.history.append({
        'iteration': opt_state.iteration,
        'loss': current_loss,
        'dt_grandchildren': current_dt_grandchildren.copy()
    })
    opt_state.iteration += 1

logging.info("Определены функции для оптимизации встречи внуков.")


In [ ]:
### 5. Запуск оптимизации

# Сбрасываем состояние перед новым запуском
opt_state = OptimizationState()

# Получаем dt для детей (они фиксированы)
fixed_dt_children = spore_tree_config.get_default_dt_vector()[:num_children]
# Начальные dt для внуков берем из конфига
initial_dt_grandchildren = spore_tree_config.get_default_dt_vector()[num_children:]

# Добавляем небольшой шум, чтобы избежать полной симметрии
np.random.seed(42)
initial_dt_grandchildren += np.random.uniform(-0.001, 0.001, size=initial_dt_grandchildren.shape)

bounds = [(0.001, 0.2)] * num_params

# Обертка для передачи фиксированных dt детей в целевую функцию
objective_wrapped = lambda dt_gc: grandchildren_pairing_loss(dt_gc, fixed_dt_children)

logging.info("--- НАЧАЛО ОПТИМИЗАЦИИ ВНУКОВ ---")
logging.info(f"Фиксированные dt детей: {np.round(fixed_dt_children, 6)}")
logging.info(f"Начальные dt внуков (с шумом): {np.round(initial_dt_grandchildren, 6)}")

# Сохраняем начальное состояние для визуализации
initial_dt_all = np.concatenate([fixed_dt_children, initial_dt_grandchildren])
evaluator._build_if_needed(initial_dt_all)
initial_tree_state = {
    'root': tree.root.copy(),
    'children': [c.copy() for c in tree.children],
    'grandchildren': [gc.copy() for gc in tree.grandchildren]
}

result = minimize(
    objective_wrapped,
    initial_dt_grandchildren,
    method=opt_config.get('method', 'L-BFGS-B'), # Используем метод из конфига или дефолтный
    bounds=bounds,
    callback=callback_function,
    options={
        'maxiter': opt_config.get('max_iters', 100),
        'disp': True,
        'ftol': float(opt_config.get('tol', 1e-7)),
        'eps': 1e-8
    }
)

logging.info("--- ОПТИМИЗАЦИЯ ВНУКОВ ЗАВЕРШЕНА ---")
logging.info(f"Сообщение: {result.message}")
logging.info(f"Финальные dt внуков: {np.round(result.x, 6)}")
logging.info(f"Финальное значение функции потерь: {result.fun:.6f}")



In [ ]:
### 6. Анализ и визуализация истории потерь

history_df = pd.DataFrame(opt_state.history)

if history_df.empty:
    logging.warning("История пуста. График не будет построен.")
else:
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(16, 8))
    
    ax.plot(history_df['iteration'], history_df['loss'], marker='o', linestyle='-', label='Loss (Pairwise Distance)')
    ax.set_title('История оптимизации потерь', fontsize=16)
    ax.set_xlabel('Итерация')
    ax.set_ylabel('Значение функции потерь')
    ax.legend()
    ax.grid(True, which="both", ls="--")
    
    # Можно использовать логарифмическую шкалу, если значения сильно меняются
    # ax.set_yscale('log')
    
    plt.tight_layout()
    plt.show()


In [ ]:
### 7. Сравнительная визуализация "до" и "после"

if 'result' in locals() and result.success and 'initial_tree_state' in locals():
    # Создаем фигуру с двумя графиками
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 11))
    
    # --- График 1: Начальное состояние ---
    visualize_spore_tree(
        initial_tree_state, 
        title="Начальное состояние (до оптимизации)",
        ax=ax1,
        show_legend=False
    )
    
    # --- График 2: Финальное состояние ---
    final_dt_grandchildren = result.x
    final_dt_all = np.concatenate([fixed_dt_children, final_dt_grandchildren])
    
    # Пересчитываем дерево с финальными dt
    evaluator._build_if_needed(final_dt_all)
    
    visualize_spore_tree(
        tree, 
        title=f"Финальное состояние после {result.nit} итераций",
        ax=ax2,
        show_legend=True
    )
    
    plt.tight_layout(rect=[0, 0, 0.9, 1])
    plt.show()
    
else:
    logging.warning("Оптимизация не была успешно завершена или начальное состояние не сохранено. "
                    "Финальная визуализация пропущена.")
